# Feedback Controls
### Tutorial on Feedback Systems (part 1 of N)
#### R. X Adhikari (5/2020)

In [ ]:

import numpy as np

import matplotlib.pyplot as plt
from scipy import signal as sig
#from scipy import interpolate
#from scipy import optimize
#import astropy.constants as const
import control
from control.matlab import *
#from timeit import default_timer as timer  # this is for timing the ODE solvers
from IPython.display import Image, SVG, display
# uncomment if you have a Mac with Retina display
#%config InlineBackend.figure_format = 'retina'
plt.style.use('dark_background')
plt.rcParams.update({'text.usetex': False,
                     'lines.linewidth': 4,
                     'font.family': 'serif',
                     'font.serif': 'Georgia',
                     'font.size': 22,
                     'xtick.direction': 'in',
                     'ytick.direction': 'in',
                     'xtick.labelsize': 'medium',
                     'ytick.labelsize': 'medium',
                     'axes.labelsize': 'medium',
                     'axes.titlesize': 'medium',
                     'axes.grid.axis': 'both',
                     'axes.grid.which': 'both',
                     'axes.grid': True,
                     'grid.color': 'xkcd:beige',
                     'grid.alpha': 0.253,
                     'lines.markersize': 12,
                     'legend.borderpad': 0.2,
                     'legend.fancybox': True,
                     'legend.fontsize': 'small',
                     'legend.framealpha': 0.8,
                     'legend.handletextpad': 0.5,
                     'legend.labelspacing': 0.33,
                     'legend.loc': 'best',
                     'figure.figsize': ((12, 8)),
                     'savefig.dpi': 140,
                     'savefig.bbox': 'tight',
                     'pdf.compression': 9})

def multZPG(P, C, A):
    # multiply them together by concatenating the zeros and poles
    pos = np.hstack((P.poles, C.poles, A.poles))
    zos = np.hstack((P.zeros, C.zeros, A.zeros))
    G = P.gain * C.gain * A.gain
    return sig.ZerosPolesGain(zos, pos, G)

## Outline
1. Linear Systems
1. Laplace Domain
1. Feedback Loops
    1. Nomenclature
    1. How to calculate TFs for SISO
    1. MIMO calculations
        1. Adjacency Matrix
1. Noise Analysis
1. Range Analysis
1. Stability Analysis
1. Optimal Feedback

## Linear Systems

## Laplace Domain

## Feedback Loops
Recommend [Astrom and Murray (2009](https://www.cds.caltech.edu/~murray/amwiki) for a nice introduction. Here lets define some terms to get started:
* **Open Loop Gain (OLG):** once around the open loop: $$OLG = P \times C \times A$$
* **Closed Loop Gain (CLG):** $$CLG = \frac{1}{1 - OLG}$$
* Forward Loop Gain
* Sensitivity
* Return something
* **Plant:** this is the physical system that we ware trying to control (e.g. a the position of a pendulum, the temperature of a box, the frequency of a quartz oscillator, etc.
* **Actuator:** this is what we use to change the plant (e.g. a force, a heat source, a variable capacitor, etc.)
* **Controller:** this is where the magic happens - this is where the feedback loop transfer function is implemented. IF its digital, the number of poles & zeros here can be very large $\sim O(100)$
* **Error Signal ($\epsilon$):** the input to the feedback Controller $C(s)$
* **Control Signal ($u$):** the out of the feedback controller (e.g. the voltage that drives the actuator)
* Dynamic Range

In [ ]:
SVG(filename='SISO-loop.svg')

In [ ]:
f = np.logspace(-1,3, 300)
w = 2*np.pi * f

p1 = 1 * np.exp(1j * 88*np.pi/180)
pend = -2*np.pi*np.array([p1, np.conj(p1)])
P = zpk2ss([], pend, 1)
P = ss(P[0], P[1], P[2], P[3])

C = zpk2ss([-2*np.pi*30, -2*np.pi*30], [0, -2*np.pi*1000], 33e9)
C = ss(C[0], C[1], C[2], C[3])

A = zpk2ss([],[-2*np.pi*1111],1)
A = ss(A[0], A[1], A[2], A[3])

# multiply them together by concatenating the zeros and poles
olg = P * C * A

clg = feedback(1, olg)

#w, mag, phase = sig.bode(olg, w=w)
mag_olg, pha_olg, _ = freqresp(olg, omega=w)
mag_clg, pha_clg, _ = freqresp(clg, omega=w)

#hclg = 1 / (1 - holg)

fig,ax = plt.subplots(2, 2, figsize=(19,10), sharex='col')

ax[0,0].semilogx(f, 20*np.log10(mag_clg.flatten()), label='CLG')
ax[0,0].semilogx(f, 20*np.log10(mag_olg.flatten()), label='OLG')
ax[0,0].set_ylim([-60, 101])

ax[0,0].set_ylabel('Mag [dB]')
ax[0,0].set_xticklabels([])
ax[0,0].legend()

ax[1,0].semilogx(f, 180/np.pi*pha_olg.flatten())
ax[1,0].semilogx(f, 180/np.pi*pha_clg.flatten())

ax[1,0].set_xlabel('Freq [Hz]')
ax[1,0].set_ylabel('Phase [deg]')
ax[1,0].set_ylim([-180, 181])
ax[1,0].set_yticks(np.arange(-180, 181, 60))

T = np.linspace(0, .031, 300)
t, y = control.impulse_response(clg)
ax[0,1].plot(t, y, label='Impulse Response')

t, y = control.step_response(clg)
ax[1,1].plot(t, y, label='Step Response')
ax[1,1].set_xlabel('Time [s]')

plt.show()

## Noise Analysis
#### How to calculate the TF from the noise injection points to elsewhere in the system:
Simple - 
1. The loop suppresses the noise so that right after the injection point: $$in\_loop noise = noise \times CLG$$
1. From that point multiple by the forward loop gain between the injection point and the point of interest.

#### Example
* How big is the contribution of the force noise "Disturb" in the error signal?

$$\epsilon = Disturb \frac{P}{1-OLG}$$ 

## Range Analysis

## Stability Analysis

## Optimal Feedback